In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/silkair/?sortby=post_date%3ADesc&pagesize=100'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    try:
        for name, rating in zip(names[1:], ratings):
            review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    except:
        review_dict[name.text] = np.nan
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'silkair'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']


# Save to CSV
df.to_csv('Data/silkair.csv',index=False)

In [3]:
# Sanity check
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
95,silkair,Bradley Baker,2015-07-06,Our flight from Singapore to Kuala Lumpar was ...,Family Leisure,Economy Class,9,5,5,3,4,5,5,yes
96,silkair,J Rui,2015-06-18,Check-in counters opened 2 hours before the fl...,Solo Leisure,Economy Class,9,4,5,4,NaN,5,4,yes
97,silkair,Sebastian T Leahy,2015-05-31,I went on MI599 bound for Kalibo. The service ...,NaN,Economy Class,9,5,4,4,2,NaN,4,yes
98,silkair,G Brambilla,2015-05-20,SIN-KUL economy (MI342) 13 May 2015. I took th...,NaN,Economy Class,6,NaN,NaN,NaN,NaN,NaN,3,yes
99,silkair,P Lewis,2015-04-14,Singapore to Siem Reap in Economy. Decent A320...,NaN,Economy Class,9,NaN,NaN,NaN,NaN,NaN,4,yes


In [4]:
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,silkair,R Ward,2020-04-02,✅ Trip Verified | Hiroshima to Kuala Lumpur v...,Couple Leisure,Economy Class,10,NaN,NaN,NaN,NaN,NaN,5,yes
1,silkair,Larry Reed,2020-03-27,✅ Trip Verified | Singapore to Yangon. On-time...,Couple Leisure,Economy Class,10,4,5,3,4,1,5,yes
2,silkair,W Manarlo,2020-03-07,✅ Trip Verified | Makassar to Singapore. Cro...,Family Leisure,Economy Class,8,4,5,5,4,3,4,yes
3,silkair,S Gan,2020-02-29,✅ Trip Verified | Am supposed to be on a flig...,Couple Leisure,Economy Class,2,3,2,1,1,2,2,no
4,silkair,Cornelius Florin,2020-02-25,✅ Trip Verified | Singapore to Davao. Pretty ...,Couple Leisure,Economy Class,9,3,5,5,1,4,4,yes
